# LangChain结构化输出

很多情况下，需要对LLM的输出做解析、验证。LangChain提供多种方式来控制Agent的输出（使用 response_format 控制代理返回结构化数据的方式）。

- ToolStrategy[StructuredResponseT]:使用工具调用进行结构化输出
- ProviderStrategy[StructuredResponseT]:使用模型提供商原生的结构化输出
- type[StructuredResponseT]:模式类型 - 根据模型能力自动选择最佳策略
- None:没有指定，采用默认模式。默认下，如果模型支持原生结构化输出，那么采用LLM提供的，如果不支持，使用ToolStrategy

样例

In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    model="deepseek-chat"
)

from pydantic import BaseModel, Field
from typing import Literal
class ProductReview(BaseModel):
    """Analysis of a product review."""
    rating: int | None = Field(description="The rating of the product", ge=1, le=5)
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")
    key_points: list[str] = Field(description="The key points of the review. Lowercase, 1-3 words each.")

from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
agent = create_agent(
    model="deepseek-chat",
    response_format=ToolStrategy(ProductReview)
)
result = agent.invoke({
    "messages": [{"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
print(result["structured_response"])

rating=5 sentiment='positive' key_points=['fast shipping', 'expensive']


## 自定义工具消息内容

tool_message_content 参数允许您自定义在生成结构化输出时出现在对话历史记录中的消息。The content of the tool message to be returned when the model calls
an artificial structured output tool.

## 错误处理

模型在生成结构化输出时可能会出错。LangChain 提供了智能重试机制来自动处理这些错误。可以使用 handle_errors 参数自定义错误处理方式：

```python
ToolStrategy(
    schema=ProductRating,
    handle_errors="Please provide a valid rating between 1-5 and include a comment."
)
```
如果 handle_errors 是字符串，则代理将始终提示模型使用固定的工具消息重试

**仅处理特定异常**
```python
ToolStrategy(
    schema=ProductRating,
    handle_errors=ValueError  # Only retry on ValueError, raise others
)
```

如果 handle_errors 是异常类型，则代理仅会在引发的异常是指定的类型时重试（使用默认错误消息）。在所有其他情况下，将引发异常。

**处理多种异常类型：**
```python
ToolStrategy(
    schema=ProductRating,
    handle_errors=(ValueError, TypeError)
)
```

如果 handle_errors 是异常元组，则代理仅会在引发的异常是指定类型之一时重试（使用默认错误消息）。在所有其他情况下，将引发异常

**自定义异常处理**

```python
def custom_error_handler(error: Exception) -> str:
    if isinstance(error, StructuredOutputValidationError):
        return "There was an issue with the format. Try again."
    elif isinstance(error, MultipleStructuredOutputsError):
        return "Multiple structured outputs were returned. Pick the most relevant one."
    else:
        return f"Error: {str(error)}"

...
handle_errors=custom_error_handler
...
```

如果不需要对错误进行处理，可以设置handle_errors=False
